# Finite state automata

## Recognizing strings with deterministic and non-deterministic FSAs

by Koenraad De Smedt at UiB

---
An *automaton* is a formal computational device that recognizes strings belonging to a formal language.

The descriptive capacity of *finite state automata* is equivalent to that of *regular expressions*. See also the notebooks on regular expressions and read ➜ Jurafsky & Martin. *Speech and Language Processing*, 3rd ed. [Ch. 2: Regular Expressions, Text Normalization, Edit Distance](https://web.stanford.edu/~jurafsky/slp3/2.pdf).

This notebook's purpose is to show how an FSA works by means of Python implementation for accepting strings with the following types of automata.

1.   deterministic FSA
2.   non-deterministic FSA

---

## 1. Deterministic FSA

A deterministic FSA can be described as follows.

1.  Q = a set of states, e.g. {q0, q1, q2, q3, ...}
2.  Σ = an input alphabet of symbols, e.g. {a, b, c, ...}
3.  q0 = an initial state q0 ∈ Q
4.  F = a set of final states F ⊆ Q, e.g. {q2, q3, ...}
5.  δ = a transition function between states δ : Q × Σ → Q

A string is accepted by an FSA if, starting from q0, successive symbols are read and each time a transition with that label is followed to a new state, until a final state is reached and the input is exhausted.

Consider the following diagram of a simple deterministic finite state automaton that recognizes the language represented by the regular expression `a+b?` , in other words, at last one (but possibly infinitely many) *a*’s followed by one *b*.

<img src="https://git.app.uib.no/desmedt/teaching/-/raw/main/dfsa-ab.png" alt = "dfsa" width = 450px>

Symbols that are input to an FSA can be represented as characters in a string. Transitions can be represented in a dict. With every state in that dict, we associate another dict: every symbol that can be read from that state has the new state for that symbol.
Also states without any departing transitions must be in the dict.

In [1]:
dfsa1 = {'q0': {'a':'q1'},
         'q1': {'a':'q1','b':'q2'},
         'q2': {}}

Here is a function that attempts to accept a string given a deterministic FSA. As arguments, the function takes a string of symbols, a dict with transitions, an initial state and a set of final states.

In a for-loop, the function prints out every successive symbol that is read from the input string, as well as the new state that is reached, if any. If no transition is found, the function returns `False`. When all symbols are read, the result is `True` if a final state is reached, `False` otherwise.

In [11]:
def accepts(s, trans, state, final):
  print('Initial state is', state)
  for c in s:
    print('Reading', c)
    state = trans[state].get(c)
    if state:
      print('- moving to state', state)
    else:
      print('- no new state found')
      return False
  return state in final

Call the function with a string of symbols, a transition dict, an initial state and a set of final states. Here are some tests.

In [12]:
accepts('aa', dfsa1, 'q0', {'q1','q2'})

Initial state is q0
Reading a
- moving to state q1
Reading a
- moving to state q1


True

In [10]:
accepts('aaab', dfsa1, 'q0', {'q1','q2'})

- moving to state q1
- moving to state q1
- moving to state q1
- moving to state q2


True

In [5]:
accepts('aba', dfsa1, 'q0', {'q1','q2'})

Initial state is q0
Reading a
- moving to state q1
Reading b
- moving to state q2
Reading a
- no new state found


False

In [6]:
accepts('b', dfsa1, 'q0', {'q1','q2'})

Initial state is q0
Reading b
- no new state found


False

The following is my recursive solution, which is a bit shorter and more elegant. You can do the same tests as above.

In [ ]:
def accepts (s, trans, state, final):
  if not state:
    return False
  print('State is', state, '- string is', s)
  if not s:
    return state in final
  return accepts(s[1:], trans, trans[state].get(s[0]), final)

## 2. Non-deterministic FSA

In the non-deterministic variant, the transition function is defined as follows.

δ : Q × Σ → 𝒫(Q)

In other words, the function gives a set of new states (subset of the power set of Q). Consider the following non-deterministic variant for the same language.

<img src="https://git.app.uib.no/desmedt/teaching/-/raw/main/nfsa-ab.png" alt = "nfsa-ab" width = 450px>

From a state, several transitions with the same symbol can depart. From *q0* in this automaton, reading symbol *a* gives two new nodes, *q0* and *q1*. In the dict, we will therefore require that with each symbol, a *list* of new states will be associated. (We could use a *set* of new states, but lists are easier to search sequentially).

In [ ]:
nfsa1 = {'q0': {'a':['q1', 'q0']},
         'q1': {'b':['q2']},
         'q2': {}}

With such non-determinism, we cannot simply proceed from one state to the next. Instead, we need to try moving from several states onwards.

Therefore we need not only a recursion over the string of symbols, but also a recursion over a list of states. A new recursive helper function `accepts_l` takes a *list* of states instead of a single state for its third parameter. Note that the `or` operator is used to try two things: first try the first state; if that is not successful, try the remaining states.

Then redefine `accepts` so that it puts the initial state in a list when it calls `accepts_l`.

In [ ]:
def accepts_l (s, trans, states, final):
  if not states:
    return False
  print('States are', states, '- string is', s)
  if not s:
    return set(states) & final # intersection non-empty?
  else:
    return (accepts_l (s[1:], trans, trans[states[0]].get(s[0]), final)
            or accepts_l (s, trans, states[1:], final))

def accepts (s, trans, state, final):
  return accepts_l(s, trans, [state], final)

The new `accepts` function can be used in the same way as the old one, except that we give it a FSA in the new format which is potentially non-deterministic. The result, if not `False`, will consist of any current nodes which are also final states.

In [ ]:
accepts('aa', nfsa1, 'q0', {'q1', 'q2'})

In [ ]:
accepts('aaab', nfsa1, 'q0', {'q1', 'q2'})

In [ ]:
accepts('aba', nfsa1, 'q0', {'q1', 'q2'})

In [ ]:
accepts('b', nfsa1, 'q0', {'q1', 'q2'})

In conclusion, a FSA (and many other kinds of automata and grammars) are formal devices that can recognize infinite languages by infinite means.

A note on *infinity*: Read about [Hilbert's Hotel](https://www.ias.edu/ideas/2016/pires-hilbert-hotel).

### Exercises

1.  Write a deterministic FSA for the regexp `a+[bc]+` and test acceptance of several strings.
2.  Extend the non-deterministic FSA to `a+b+` by adding a loop with *b* at *q1*, which causes additional non-determinism. Perform tests.
3.  (optional) Write a function that converts a representation of transitions as shown in "1. Deterministic FSA" to the format shown in "2. Non-deterministic FSA". This can be a one-liner, if you use a double *dict comprehension*.